In [42]:
import numpy as np

np.random.seed(0)
np.set_printoptions(precision=2)

In [46]:
def generate_matrix(n):
    M = np.random.randint(low=1, high=10, size=(n,n))
    return M.astype(np.float64)

A = generate_matrix(5)
print(A)

[[6. 1. 3. 4. 9.]
 [2. 4. 4. 4. 8.]
 [1. 2. 1. 5. 8.]
 [4. 3. 8. 3. 1.]
 [1. 5. 6. 6. 7.]]


In [44]:
def lu(A):
    """Return the LU decomposition of A, without modifying it."""
    if not (n := A.shape[0]) == A.shape[1]:
        raise ValueError("Input matrix must be square.")

    U = np.copy(A)
    L = np.identity(n, dtype=np.float64)

    for col in range(n-1):
        for row in range(col + 1, n):
            L[row, col] = U[row, col] / U[col, col]
            U[row, col:] -= L[row, col] * U[col, col:]
            U[row, col] = 0

    return (L, U)

In [45]:
L, U = lu(A)
assert np.allclose(L @ U, A)
print(L)
print(U)

[[ 1.    0.    0.    0.    0.  ]
 [ 0.67  1.    0.    0.    0.  ]
 [ 1.17  1.47  1.    0.    0.  ]
 [ 1.33  1.25  0.85  1.    0.  ]
 [ 1.5   0.66 -0.58  9.16  1.  ]]
[[ 6.    1.    4.    4.    8.  ]
 [ 0.    5.33  0.33  2.33  2.67]
 [ 0.    0.    3.84 -6.09 -6.25]
 [ 0.    0.    0.   -1.1  -2.72]
 [ 0.    0.    0.    0.   11.5 ]]
